# Segmenting and Clustering Neighborhoods in Toronto -Part 2

##  This notebook includes part 1 code to run independantly. The main objective for this notebook is to get geographical coordinates from https://cocl.us/Geospatial_data ， and generate dataframe for furthur analysis usage. 


###  Due to my windows environment, some libraries were installed locally using Anaconda.
### geopy, folium, wget, curl and json were installed with Anacoda locally.  


In [58]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import geocoder # import geocoder

#!conda install -c conda-forge geopy --yes 

# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans
import folium # map rendering library
import requests
import geopy.geocoders

from lxml import html
print('Libraries imported.')


Libraries imported.


In [59]:
df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#results=requests.get(url)
df=pd.read_html(url)
tb = pd.DataFrame(columns=['Postal Code', 'Borough', 'Neighborhood'])
tb=pd.read_html(url,header=0)[0]
#tb.to_csv('./output.csv',mode='a', encoding='utf-8', header=0, index=0)
#df.head(6)
tb.head(10)



,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [60]:
tb_clean=tb[tb.Borough !='Not assigned']

pd.set_option('display.width',2000)

print(tb_clean)




    Postcode           Borough                                      Neighbourhood
2        M3A        North York                                          Parkwoods
3        M4A        North York                                   Victoria Village
4        M5A  Downtown Toronto                                       Harbourfront
5        M5A  Downtown Toronto                                        Regent Park
6        M6A        North York                                   Lawrence Heights
7        M6A        North York                                     Lawrence Manor
8        M7A      Queen's Park                                       Not assigned
10       M9A         Etobicoke                                   Islington Avenue
11       M1B       Scarborough                                              Rouge
12       M1B       Scarborough                                            Malvern
14       M3B        North York                                    Don Mills North
15       M4B    

In [61]:
tb_clean_merge=tb_clean.groupby(['Postcode','Borough']).apply(lambda x:[','.join(x['Neighbourhood'])])
#tb_clean_merge=tb_clean['Postcode'].groupby('Borough').apply(lambda x:[','.join(x['Neighbourhood'])])
#tb_clean_merge.head(10)

#
tb_clean_merge=tb_clean_merge.to_frame().reset_index()
#tb_clean_merge.rename(columns={"Postcode","Borough","Neighbourbood"})
tb_clean_merge.columns=['Postal Code','Borough','Neighbourhood']



#type(tb_clean_merge)
print(tb_clean_merge)

tb_clean_merge.shape

    Postal Code           Borough                                      Neighbourhood
0           M1B       Scarborough                                    [Rouge,Malvern]
1           M1C       Scarborough             [Highland Creek,Rouge Hill,Port Union]
2           M1E       Scarborough                  [Guildwood,Morningside,West Hill]
3           M1G       Scarborough                                           [Woburn]
4           M1H       Scarborough                                        [Cedarbrae]
5           M1J       Scarborough                              [Scarborough Village]
6           M1K       Scarborough        [East Birchmount Park,Ionview,Kennedy Park]
7           M1L       Scarborough                    [Clairlea,Golden Mile,Oakridge]
8           M1M       Scarborough    [Cliffcrest,Cliffside,Scarborough Village West]
9           M1N       Scarborough                       [Birch Cliff,Cliffside West]
10          M1P       Scarborough  [Dorset Park,Scarborough Town 

(103, 3)

## Part 2 code starting from here.

### Download the Toronto data geo data as csv file. The Geocoder API is very unreliable and won't be able to return data. 

In [63]:


!curl -s -L -o Toronto.csv https://cocl.us/Geospatial_data
print('Toronto GEO Data donloaded!')
toronto_data = pd.read_csv('./Toronto.csv') 






Toronto GEO Data donloaded!


### Sanity checking the downloaded data.

In [64]:
toronto_data.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### Pre-flight checking and saved the results locally for debugging purpose.

In [65]:


#Pre-flight checking on tb_clean_merge
tb_clean_merge.to_csv('./tb_clean_merge.csv',mode='a', encoding='utf-8', header=0, index=0)
#Pre-flight checking on toronto_data
toronto_data.to_csv('./toronto_data.csv',mode='a', encoding='utf-8', header=0, index=0)




### Merging the two data frames to include postal code, Borough, Neighborhood, Latitude and Longitude.


In [69]:
df_final= pd.merge(tb_clean_merge, toronto_data, how='left', on='Postal Code')
print(df_final)

    Postal Code           Borough                                      Neighbourhood   Latitude  Longitude
0           M1B       Scarborough                                    [Rouge,Malvern]  43.806686 -79.194353
1           M1C       Scarborough             [Highland Creek,Rouge Hill,Port Union]  43.784535 -79.160497
2           M1E       Scarborough                  [Guildwood,Morningside,West Hill]  43.763573 -79.188711
3           M1G       Scarborough                                           [Woburn]  43.770992 -79.216917
4           M1H       Scarborough                                        [Cedarbrae]  43.773136 -79.239476
5           M1J       Scarborough                              [Scarborough Village]  43.744734 -79.239476
6           M1K       Scarborough        [East Birchmount Park,Ionview,Kennedy Park]  43.727929 -79.262029
7           M1L       Scarborough                    [Clairlea,Golden Mile,Oakridge]  43.711112 -79.284577
8           M1M       Scarborough    

In [73]:

df_final.head()
df_final.to_csv('./df_final_data.csv',mode='a', encoding='utf-8', header=0, index=0)

In [74]:
df_final.dtypes

Postal Code       object
Borough           object
Neighbourhood     object
Latitude         float64
Longitude        float64
dtype: object

### Snaity check the Toronto city coordinate. 

In [75]:
# initialize your variable to None
lat_lng_coords = None


#address='M5G'
address = 'Toronto ON'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
print(location)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))


# loop until you get the coordinates
#while(lat_lng_coords is None):
  #g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
 # g = geocoder.google('Mountain View, CA')
  #print('check Point Inside loop', lat_lng_coords)
  #lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

Toronto, Golden Horseshoe, Ontario, M6K 1X9, Canada
The geograpical coordinate of Toronto City are 43.653963, -79.387207.
